In [1]:
import torch

from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)

net(X)

tensor([[-0.1362, -0.0142,  0.2712, -0.1053,  0.2453, -0.1216,  0.2030,  0.0732,
         -0.0179,  0.1814],
        [-0.0049,  0.1157,  0.2130, -0.1058,  0.1011, -0.1384,  0.2332,  0.1173,
          0.0364,  0.2392]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self. out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 0.1663,  0.0513, -0.0829, -0.0624, -0.0385,  0.1807, -0.0222, -0.2307,
          0.0782, -0.0037],
        [ 0.1405,  0.0998,  0.0200, -0.1054,  0.0441,  0.1779, -0.1293, -0.2563,
          0.0077,  0.0631]], grad_fn=<AddmmBackward0>)

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
   
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        
        return X

net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

net(X)


tensor([[ 0.0493, -0.0828,  0.2007,  0.1561,  0.1809,  0.0290, -0.2270,  0.1423,
         -0.1494, -0.3247],
        [ 0.0503, -0.0582,  0.1621,  0.0366,  0.1587,  0.1476, -0.1034,  0.1402,
         -0.1252, -0.2345]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.4308, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))
    
net = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
net(X)

tensor(0.1335, grad_fn=<SumBackward0>)

In [8]:
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

X = torch.rand(size = (2,4))
net(X)

tensor([[0.1979],
        [0.1298]], grad_fn=<AddmmBackward0>)

In [9]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1405,  0.0157, -0.0212, -0.3375,  0.1628, -0.2270, -0.0630, -0.1698]])), ('bias', tensor([0.3162]))])


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3162], requires_grad=True)
tensor([0.3162])
None


In [11]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
net.state_dict()['2.bias'].data

tensor([0.3162])

In [13]:
def block1():
    return nn.Sequential(nn.Linear(4,8),
                         nn.ReLU(),
                         nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(
    block2(),
    nn.Linear(4,1)
)
rgnet(X)
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [14]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0076,  0.0044, -0.0045,  0.0003]), tensor(0.))

In [15]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [16]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0419,  0.6642,  0.5316, -0.2347])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [17]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -6.2898, -0.0000],
        [ 0.0000,  0.0000, -7.3777, -0.0000]], grad_fn=<SliceBackward0>)

In [18]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -5.2898,  1.0000])

In [19]:
# Link parameters, share weights
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8,1)
)

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 52
print(net[4].weight.data[0, 0])
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor(52.)
tensor([True, True, True, True, True, True, True, True])


In [21]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [22]:
net = nn.Sequential(
    nn.Linear(8, 128),
    CenteredLayer()
)

Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.0245e-08, grad_fn=<MeanBackward0>)

In [23]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-1.0064, -0.5660,  0.1190],
        [ 0.6000,  0.3548,  0.2642],
        [-0.4160,  2.0041,  1.9512],
        [-0.9376,  1.5867, -0.8665],
        [-2.4565,  0.7637, -1.0907]], requires_grad=True)

In [24]:
dense(torch.rand(2, 5))

tensor([[0.0000, 2.6739, 1.4392],
        [0.0000, 1.7879, 0.9644]])

In [25]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[10.5452],
        [12.8009]])

In [26]:
x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [30]:
y = torch.zeros(4)
torch.save([x, y], 'draft/x-files')

x2, y2 = torch.load('draft/x-files')
(x2, y2)


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [31]:
mydict = {'x':x, 'y':y}
torch.save(mydict, 'draft/mydict')
mydict2 = torch.load('draft/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [33]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [34]:
torch.save(net.state_dict(), 'draft/mlp.params')

In [36]:
clone = MLP()
clone.load_state_dict(torch.load('draft/mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [37]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])